<a href="https://colab.research.google.com/github/kaustubh-Beta/DeepLearning/blob/master/data_loading_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!echo "Downloading 256_Object_Categories for image notebooks"
!curl -L -o 256_ObjectCategories.tar.gz --progress-bar http://www.vision.caltech.edu/Image_Datasets/Caltech256/256_ObjectCategories.tar
!tar -xzf 256_ObjectCategories.tar.gz
!rm 256_ObjectCategories.tar.gz
!ls

tar (child): 256_ObjectCategories.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
rm: cannot remove '256_ObjectCategories.tar.gz': No such file or directory
sample_data


In [0]:
!mkdir data

In [0]:
%matplotlib inline

In [0]:
import numpy as np
import cv2
import matplotlib.pyploy as plt

